In [8]:
from google.colab import drive
import os
import pandas as pd

# 1) 구글 드라이브 마운트 및 경로 설정
drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/aladin-project'
os.chdir(project_path)

# 2) 파일 로드
df_aladin = pd.read_csv('data/raw/aladin.csv', index_col=0, dtype={'item_id': str})
df_detail = pd.read_csv('data/raw/detail_mapping.csv', dtype={'item_id': str})

print("aladin.csv 로드 완료")
print("detail_mapping.csv 로드 완료")
print("=" * 50)

# 3) 병합 (기본 베스트셀러 + 상세데이터)
df_merged = pd.merge(
    df_aladin,
    df_detail.drop_duplicates(subset=['item_id']),   # 상세정보 중복 제거
    on='item_id',
    how='left',
    suffixes=('', '_new')
)

# 4) 카테고리 업데이트 (기본 category ← 상세 real_category)
has_new_category = df_merged['real_category'].notnull() & (df_merged['real_category'] != 'N/A')
df_merged.loc[has_new_category, 'category'] = df_merged.loc[has_new_category, 'real_category']

# 상세 카테고리 업데이트 안 된 경우 계산
failed_category_update = (~has_new_category).sum()

print(f"[카테고리 업데이트 성공] {has_new_category.sum()}개")
print(f"[카테고리 업데이트 실패] {failed_category_update}개 (여전히 '국내도서' 상태)")
print("=" * 50)

# real_category 컬럼 제거
df_merged = df_merged.drop(columns=['real_category'])

# 5) 페이지 수 업데이트 (기본 page_count ← 상세 page_count_new)
if 'page_count_new' in df_merged.columns:
    df_merged['page_count'] = df_merged['page_count_new'].fillna(df_merged['page_count'])
    df_merged = df_merged.drop(columns=['page_count_new'])

updated_pages = (df_merged['page_count'] > 0).sum()
print(f"[페이지 업데이트] 1쪽 이상 페이지 데이터: {updated_pages}개")
print("=" * 50)

# 6) 페이지가 0인 도서 삭제 (결측치 제거)
condition_page_zero = (df_merged['page_count'] == 0)
num_page_zero = condition_page_zero.sum()
df_cleaned = df_merged[~condition_page_zero]

print(f"[삭제] 페이지 수 0인 도서: {num_page_zero}개 제거됨")
print(f"[최종 데이터] {len(df_cleaned)}개 행 유지")
print("=" * 50)

# 7) 저장
os.makedirs('data/processed', exist_ok=True)
df_cleaned.to_csv('data/processed/aladin_final_cleaned.csv', encoding='utf-8-sig', index=True)

print("✔ 최종 파일 저장 완료: data/processed/aladin_final_cleaned.csv")

print(f"- 최종 행 수: {len(df_cleaned)}")
print(f"- 고유 도서: {df_cleaned['item_id'].nunique()}개")
print(f"- 페이지 수 존재 도서: {(df_cleaned['page_count'] > 0).sum()}개")
print(f"- 상세 카테고리로 업데이트되지 못한 행: {failed_category_update}개")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
aladin.csv 로드 완료
detail_mapping.csv 로드 완료
[카테고리 업데이트 성공] 3517개
[카테고리 업데이트 실패] 22개 (여전히 '국내도서' 상태)
[페이지 업데이트] 1쪽 이상 페이지 데이터: 3517개
[삭제] 페이지 수 0인 도서: 22개 제거됨
[최종 데이터] 3517개 행 유지
✔ 최종 파일 저장 완료: data/processed/aladin_final_cleaned.csv
- 최종 행 수: 3517
- 고유 도서: 1937개
- 페이지 수 존재 도서: 3517개
- 상세 카테고리로 업데이트되지 못한 행: 22개


In [9]:
import pandas as pd

# 데이터 로드
df = pd.read_csv('data/processed/aladin_final_cleaned.csv', index_col=0, dtype={'item_id': str})

# 연도별 도서 개수
yearly_count = df.groupby('year').size()

print("=" * 50)
print("연도별 베스트셀러 진입 도서 개수")
print("=" * 50)

for year, count in yearly_count.items():
    print(f"{year}년: {count:,}개")

print("=" * 50)
print(f"전체 합계: {len(df):,}개")

연도별 베스트셀러 진입 도서 개수
2020년: 597개
2021년: 594개
2022년: 599개
2023년: 585개
2024년: 598개
2025년: 544개
전체 합계: 3,517개
